In [ ]:
from grindr_access.grindrUser import grindrUser
from time import sleep
import random, os, pickle
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import numpy as np
from dotenv import load_dotenv
from copy import deepcopy
from influxdb_client import InfluxDBClient, Point, WriteOptions,WritePrecision
from mongoengine import connect
load_dotenv()

user = grindrUser()
mail = os.getenv('GRINDR_MAIL')
password = os.getenv('GRINDR_PASS')
krk_lat = 50.059185
krk_lon = 19.937809

user.login(mail, password)
connect(
    db=os.getenv('MONGO_DB'),
    host=os.getenv('MONGO_URL'),
    port=27017,
    username=os.getenv('MONGO_USR'),
    password=os.getenv('MONGO_PWD'),
    authentication_source="admin"
    )


In [ ]:
def generate_grid_points(center_lat, center_lon, side_m, points_per_side):
    # Approximate conversion factor: 1 degree latitude = 111,000 meters
    deg_per_m = 1 / 111000

    # Calculate half side in degrees
    half_side_deg = (side_m / 2) * deg_per_m

    # Define the rectangle corners
    top_lat = center_lat + half_side_deg
    bottom_lat = center_lat - half_side_deg
    # Adjust for longitude using cos(), now considering meters
    left_lon = center_lon - half_side_deg / cos(radians(center_lat))
    right_lon = center_lon + half_side_deg / cos(radians(center_lat))

    # Generate latitude and longitude points
    lat_steps = np.linspace(bottom_lat, top_lat, points_per_side)
    lon_steps = np.linspace(left_lon, right_lon, points_per_side)

    grid_points = [(lat, lon) for lat in lat_steps for lon in lon_steps]

    return grid_points


In [ ]:
# Example usage
center_lat, center_lon = krk_lat, krk_lon # Equator and Prime Meridian
side_m = 10000 # 10 km side length
accuracy_m = 1000 # 1000m 
points_per_side = int(side_m/accuracy_m) # Generate 64 points

grid_points = generate_grid_points(center_lat, center_lon, side_m, points_per_side)

In [ ]:
record_measurement_key="profileId",
record_time_key="created",
record_tag_keys=["lat", "lon","type","batch_timestamp","@type"],
record_field_keys=['lastOnline', 'photoMediaHashes', 'heightCm',
'weightGrams', 'lookingFor', 'tribes', 'meetAt', 'vaccines', 'genders',
'pronouns', 'bodyType', 'approximateDistance', 'tags', 'isFavorite',
'socialNetworks', 'isBoosting', 'hasChattedInLast24Hrs',
'hasUnviewedSpark', 'isTeleporting', 'isRightNow', 'unreadCount',
'rightNow', 'distanceMeters',
'relationshipStatus', 'aboutMe', 'age', 'ethnicity', 'sexualPosition',
'acceptsNsfwPics', 'hivStatus', 'displayName', 'lastTestedDate',
'cascadePlacementName', "lat", "lon"]

In [ ]:
from profile_model import ProfileModel

In [ ]:
i = 0
profiles = {}
batch_timestamp = int(datetime.now().timestamp() * 1000)
for point in grid_points:
    timestamp = int(datetime.now().timestamp() * 1000)
    loc_date_key = (point[0], point[1], timestamp)
    time_to_sleep = random.uniform(0, 4)
    print(f"{i}/{len(grid_points)} - {point} - {timestamp} - {time_to_sleep}")
    profile_list = user.getProfiles(point[0], point[1])
    for profile in profile_list['items']:
        prof = {}
        prof.update(profile['data'])
        profile_types = ["PartialProfileV1", "FullProfileV1"]
        if any(element in prof.get('@type') for element in profile_types):
            prof['created'] = timestamp
            prof['lat'] =  point[0]
            prof['lon'] =  point[1]
            prof['batch_timestamp'] = batch_timestamp
            new_doc = ProfileModel(**prof)
            new_doc.save()


    profiles[loc_date_key] = deepcopy(profile_list)
    sleep(time_to_sleep)
    i+=1

In [ ]:
# Ensure the data directory exists
os.makedirs('data', exist_ok=True)
# Get the current datetime in Unix time milliseconds
file_name = f"{int(datetime.now().timestamp() * 1000)}.pickle"
file_path = os.path.join('data', file_name)

# Pickle the dictionary
with open(file_path, 'wb') as file:
    pickle.dump(profiles, file)
